<a href="https://colab.research.google.com/github/aliss153/Aprendizaje-Automatico/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Alisson Manosalvas &bull; 25 Enero 2026
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [45]:
# Paquetes necesarios
from sklearn.datasets import fetch_openml
import pandas as pd  # Manejo de datos
import matplotlib.pyplot as plt  # Visualización
import joblib

from sklearn.model_selection import train_test_split # División de datos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métrica de evaluación

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold  # Búsqueda de hiperparámetros
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import KFold

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

- Muestra algunos datos

In [2]:
url_data = "https://raw.githubusercontent.com/aliss153/Aprendizaje-Automatico/main/Country-data.csv"
df = pd.read_csv(url_data)
df.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
1,Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
3,Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200


- Muestra una descripción de los datos.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     167 non-null    object 
 1   child_mort  167 non-null    float64
 2   exports     167 non-null    float64
 3   health      167 non-null    float64
 4   imports     167 non-null    float64
 5   income      167 non-null    int64  
 6   inflation   167 non-null    float64
 7   life_expec  167 non-null    float64
 8   total_fer   167 non-null    float64
 9   gdpp        167 non-null    int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 13.2+ KB


In [4]:
df.describe()

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
count,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000
mean,38.270060,41.108976,6.815689,46.890215,17144.688623,7.781832,70.555689,2.947964,12964.155689
std,40.328931,27.412010,2.746837,24.209589,19278.067698,10.570704,8.893172,1.513848,18328.704809
min,2.600000,0.109000,1.810000,0.065900,609.000000,-4.210000,32.100000,1.150000,231.000000
25%,8.250000,23.800000,4.920000,30.200000,3355.000000,1.810000,65.300000,1.795000,1330.000000
50%,19.300000,35.000000,6.320000,43.300000,9960.000000,5.390000,73.100000,2.410000,4660.000000
75%,62.100000,51.350000,8.600000,58.750000,22800.000000,10.750000,76.800000,3.880000,14050.000000
max,208.000000,200.000000,17.900000,174.000000,125000.000000,104.000000,82.800000,7.490000,105000.000000


- Escala los datos si es necesario.

In [8]:
numeric_cols = df.drop(columns=['country']).columns
numeric_cols

Index(['child_mort', 'exports', 'health', 'imports', 'income', 'inflation',
       'life_expec', 'total_fer', 'gdpp'],
      dtype='object')

In [14]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[numeric_cols])

df_scaled = pd.DataFrame(df_scaled, columns=numeric_cols)
df_scaled.head()

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,1.291532,-1.138280,0.279088,-0.082455,-0.808245,0.157336,-1.619092,1.902882,-0.679180
1,-0.538949,-0.479658,-0.097016,0.070837,-0.375369,-0.312347,0.647866,-0.859973,-0.485623
2,-0.272833,-0.099122,-0.966073,-0.641762,-0.220844,0.789274,0.670423,-0.038404,-0.465376
3,2.007808,0.775381,-1.448071,-0.165315,-0.585043,1.387054,-1.179234,2.128151,-0.516268
4,-0.695634,0.160668,-0.286894,0.497568,0.101732,-0.601749,0.704258,-0.541946,-0.041817


Creo una clase a partir de una variiable numerica, donde 0 es el país de bajo ingreso y el 2 de alto ingreso

In [26]:
df['income_class'] = pd.qcut(
    df['income'],
    q=3,
    labels=[0, 1, 2])

In [27]:
X = df_scaled
y = df['income_class']


### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [18]:
modelo_base = SVC()
modelo_base.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

**- Consulta qué significan al menos 4 hiperparámetros.**

- **c:** Controla el compromiso entre maximizar el márgen y minimizar los erores de clasificación, en donde valores altos pueden generar sobreajuste, y los más bajos producen modelos más simples

- **kernel:** Es como la frontera de decisión, puuede ser 'lineal' o no lineal ccon 'rbf' y 'poly', de maneera que permite modelar relaciones complejas entre variables

- **gamma:** Controla la influencia dee cada obvservación individual, altros valores tienen una influencia muy local con riego de sobreajuste, los valores bajos tienen una infloeuncia más global (modelo más suave), solo aplica para kernels no lineales

- **degree:** Indica el grado de polinomio cuando se usa kernel polinómico, a mayor grado, tiene mayor complejidad del modelo

- Selecciona los hiperparámetros que deseas optimizar, al menos 3.

In [29]:
parametros = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']}

In [30]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [32]:
grid_search = GridSearchCV(
    estimator=modelo_base,
    param_grid=parametros,
    cv=k_fold,
    scoring='accuracy'
)

grid_search.fit(X, y)

print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

print("\nMejor score obtenido:")
print(grid_search.best_score_)

Mejores hiperparámetros encontrados:
{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}

Mejor score obtenido:
0.9098039215686274


In [33]:
resultados = pd.DataFrame(grid_search.cv_results_)
resultados[['params', 'mean_test_score', 'std_test_score']]

,params,mean_test_score,std_test_score
0,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}",0.837790,0.050269
1,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}",0.795900,0.065896
2,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'linear'}",0.837790,0.050269
3,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}",0.801961,0.054062
4,"{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}",0.873975,0.040293
5,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",0.844029,0.062343
6,"{'C': 1, 'gamma': 'auto', 'kernel': 'linear'}",0.873975,0.040293
7,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",0.844029,0.062343
8,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.909804,0.043119
9,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.867914,0.056780


El modelo optimizado alcanza una precisión promedio del 90.98%,lo que indica un desempeño alto y buena capacidad de generalización, se obtuvieron utilizando un kernel lineal con un valor de regularización C = 10 y gamma = 'scale'

### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

- Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.

In [34]:
parametros_random = {
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1]}

- Utiliza al menos 5 validaciones cruzadas.

In [35]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

- Usa RandomSearchCV con 25 iteraciones.

In [37]:
random_search = RandomizedSearchCV(
    estimator=modelo_base,
    param_distributions=parametros_random,
    n_iter=25,
    cv=k_fold,
    scoring='accuracy',
    random_state=42)

random_search.fit(X, y)


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=SVC(), n_iter=25,
                   param_distributions={'C': [0.01, 0.1, 1, 10, 100],
                                        'gamma': ['scale', 'auto', 0.001, 0.01,
                                                  0.1],
                                        'kernel': ['linear', 'rbf', 'poly',
                                                   'sigmoid']},
                   random_state=42, scoring='accuracy')

- Muestra los parámetros óptimos y su score.

In [38]:
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

print("\nMejor score obtenido:")
print(random_search.best_score_)

Mejores hiperparámetros encontrados:
{'kernel': 'linear', 'gamma': 'scale', 'C': 100}

Mejor score obtenido:
0.9279857397504457


El modelo optimizado mediante RandomizedSearch alcanzó una precisión promedio del 92.7%, superando ligeramente el resultado obtenido con GridSearch

### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [46]:
modelo_final = SVC(
    C=100,
    kernel='linear',
    gamma='scale')
modelo_final.fit(X, y)
score_final = modelo_final.score(X, y)
print("Score del modelo final:", score_final)

Score del modelo final: 0.9880239520958084


In [47]:
joblib.dump(modelo_final, "svm_country_income_model.pkl")

['svm_country_income_model.pkl']

In [48]:
modelo_cargado = joblib.load("svm_country_income_model.pkl")
modelo_cargado


SVC(C=100, kernel='linear')

### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>

-> Listo